In [2]:
# Name:        Jordan Crisler
# Professor:   Professor Conlan
# Course:      CS-499-14507-M01 Computer Science Capstone 2025 C-6
# Date:        December 6, 2025
# Artifact 2

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configures the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

import getpass

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter ##Import animal_shelter and AnimalShelter for CRUD module

###########################
# Data Manipulation / Model
###########################

#Removes hardcoded credentials to database and requests user input for username and password
username = input("Please Enter the MongoDB Username: ") ##Requests user to enter MongoDB username
password = getpass.getpass("Please enter the MongoDB Password: ") ##Requests user to enter MongoDB password

# Connect to database via CRUD Module
db = AnimalShelter(username, password) ##Adds CRUD Python module name and value

# Class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso-Salvare-Logo.png' # Adds Grazioso Salvare's logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Places the HTML image tag in the line below into the app

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style = {'height':'200px', 'width':'200px'}),
    html.Center(html.B(html.H2('Capstone Project - Jordan Crisler'))), #Adds unique identifier heading
    html.Center(html.B(html.H1('CS-499 Dashboard'))),
    html.Hr(),
    html.Div(
        
        #Creates radio item interactive filtering option
        dcc.RadioItems(
        id='filter-type',
        options = [
            {'label':'Water Rescue', 'value': 'water'}, #Adds water rescue option
            {'label':'Mountain or Wilderness Rescue', 'value': 'mountain'}, #Adds mountain or wilderness rescue option
            {'label':'Disaster Rescue or Individual Tracking', 'value': 'disaster'}, #Adds disaster rescue and individual tracking option
            {'label':'Reset', 'value': 'reset'} #Adds reset option
        ],
        value = 'reset',
    )
    
    ),
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
# Sets up the features for interactive data table to make it user-friendly
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    #Adds queries to filter intactice data table with filter type options
    if filter_type is None:
        return []
    
    df = None
        
    if filter_type == 'water': #Filter type for water rescue option
        query = {
            "$and": [
        {
            "$or": [
                {"breed": {"$regex": "Labrador Retriever Mix"}},
                {"breed": {"$regex": "Chesapeake Bay Retriever"}},
                {"breed": {"$regex": "Newfoundland"}}
            ]
        },
        {"sex_upon_outcome": "Intact Female"},
        {
            "$and": [
                {"age_upon_outcome_in_weeks": {"$gt": 26}},
                {"age_upon_outcome_in_weeks": {"$lt": 156}}
            ]
        }
    ]
        }
        
    elif filter_type == 'mountain': #Filter type for mountain and wilderness rescue option
        query = {
                "$and": [
        {
            "$or": [
                {"breed": {"$regex": "German Shepherd"}},
                {"breed": {"$regex": "Alaskan Malamute"}},
                {"breed": {"$regex": "Old English Sheepdog"}},
                {"breed": {"$regex": "Siberian Husky"}},
                {"breed": {"$regex": "Rottweiler"}}
            ]
        },
        {"sex_upon_outcome": "Intact Male"},
        {
            "$and": [
                {"age_upon_outcome_in_weeks": {"$gt": 26}},
                {"age_upon_outcome_in_weeks": {"$lt": 156}}
            ]
        }
    ]
        }
        
    elif filter_type == 'disaster': # Filter type for disaster rescue and individual tracking option
       query = {
           "$and": [
        {
            "$or": [
                {"breed": {"$regex": "Doberman Pinscher"}},
                {"breed": {"$regex": "German Shepherd"}},
                {"breed": {"$regex": "Golden Retriever"}},
                {"breed": {"$regex": "Bloodhound"}},
                {"breed": {"$regex": "Rottweiler"}}
            ]
        },
        {"sex_upon_outcome": "Intact Male"},
        {
            "$and": [
                {"age_upon_outcome_in_weeks": {"$gt": 20}},
                {"age_upon_outcome_in_weeks": {"$lt": 300}}
            ]
        }
    ]
       }
        
    else:
        df = pd.DataFrame.from_records(db.read({"animal_type": "Dog"}))
        
    df = pd.DataFrame.from_records(db.read(query))   

    if df.empty():
        return []

    if '_id' in df.columns:
        df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')
    

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    
    df = pd.DataFrame.from_records(viewData)
        
    return [
        dcc.Graph(
            figure = px.pie(df, names= "breed", title= "Preferred Animals") 
        )
    ]


#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry derived_virtual_data 
# will be the set of data available from the datatable in the form of a dictionary. Derived_virtual_selected_rows 
# will be the selected row(s) in the table in the form of a list. For this application, we are only permitting
# single row selection so there is only one value in the list. The iloc method allows for a row, column notation 
# to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or len(viewData) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)

    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]

    # Checks to ensure row exists
    if row >= len(dff):
        row = 0

    lat = dff.iloc[row].get("location_lat", None)
    lon = dff.iloc[row].get("location_long", None)
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, 
               center=[30.75,-97.48], 
               zoom=10, 
               children=[
                    dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(dff.iloc[row,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row]["name"])
                ])
            ])
        ])
    ]



app.run(mode='external', debug=True)


Please Enter the MongoDB Username:  aacuser
Please enter the MongoDB Password:  ········


C:\Users\jcris\AppData\Roaming\Python\Python314\site-packages\dash\dash.py:642: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Error on request:
Traceback (most recent call last):
  File "C:\Users\jcris\AppData\Roaming\Python\Python314\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\jcris\AppData\Roaming\Python\Python314\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\jcris\AppData\Roaming\Python\Python314\site-packages\dash\_get_app.py", line 17, in wrap
    return ctx.run(func, self, *args, **kwargs)
           ~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jcris\AppData\Roaming\Python\Python314\site-packages\dash\dash.py", line 1585, in dispatch
    response_data = ctx.run(partial_func)
  File "C:\Users\jcris\AppData\Roaming\Python\Python314\site-packages\dash\_callback.py", line 721, in add_context
    raise err
  File "C:\Us